In [23]:
import os.path
import numpy as np
import pandas as pd
from morfeus.buried_volume import BuriedVolume
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import DrawingOptions
from parameter_extractor import ParameterExtractor, Property

IPythonConsole.molSize = (300, 300)  # Change image size
DrawingOptions.bondLineWidth = 1.8

In [2]:
filename = "l_2"
path = f"./out/spe/{filename}_SPE.out"
path_no_rh = f"./out/spe_no_rh/{filename}_SPE_NoRh.out"
if not os.path.exists(path_no_rh):
    print(path_no_rh + " does not exist")

In [3]:
comp = ParameterExtractor(path=path, no_rh_path=path_no_rh)

In [8]:
bite_angle = comp.get_angle(comp.p1, comp.rh, comp.p2)
p1_rh_dist = comp.get_bond_distance(comp.p1, comp.rh)
p2_rh_dist = comp.get_bond_distance(comp.p2, comp.rh)
p1_bridge_dist = comp.get_bond_distance(comp.p1, comp.bridge)
p2_bridge_dist = comp.get_bond_distance(comp.p2, comp.bridge)
bridge_angle = comp.get_angle(comp.p1, comp.bridge, comp.p2)

print("P1-Rh: " + str(p1_rh_dist), "P2-Rh: " + str(p2_rh_dist), "P1-Bridge: " + str(p1_bridge_dist),
      "P2-Bridge: " + str(p2_bridge_dist), "Bite angle: " + str(bite_angle), "Bridge angle: " + str(bridge_angle))

P1-Rh: 2.3719312711164293 P2-Rh: 2.37194201200198 P1-Bridge: 1.8750391887104654 P2-Bridge: 1.8749946723657644 Bite angle: 72.70956883041772 Bridge angle: 97.16063672571833


In [9]:
comp.dipole

1.9588

In [10]:
r1_p1_r2_angle = comp.get_angle(comp.r1, comp.p1, comp.r2)
r1_p1_rh_angle = comp.get_angle(comp.r1, comp.p1, comp.rh)
rh_p1_bridge_angle = comp.get_angle(comp.rh, comp.p1, comp.bridge)
r1_p1_bridge_angle = comp.get_angle(comp.r1, comp.p1, comp.bridge)
r2_p1_bridge_angle = comp.get_angle(comp.r2, comp.p1, comp.bridge)
r2_p1_rh_angle = comp.get_angle(comp.r2, comp.p1, comp.rh)

p1_angle_sum = r1_p1_bridge_angle + r1_p1_r2_angle + r2_p1_bridge_angle
p1_angle_sum

317.8066693255203

In [11]:
r3_p2_r4_angle = comp.get_angle(comp.r3, comp.p2, comp.r4)
r3_p2_rh_angle = comp.get_angle(comp.r3, comp.p2, comp.rh)
rh_p2_bridge_angle = comp.get_angle(comp.rh, comp.p2, comp.bridge)
r3_p2_bridge_angle = comp.get_angle(comp.r3, comp.p2, comp.bridge)
r4_p2_bridge_angle = comp.get_angle(comp.r4, comp.p2, comp.bridge)
r4_p2_rh_angle = comp.get_angle(comp.r4, comp.p2, comp.rh)

p2_angle_sum = r3_p2_bridge_angle + r3_p2_r4_angle + r4_p2_bridge_angle
p2_angle_sum

317.80297812394565

In [12]:
comp.mol.GetBondBetweenAtoms(comp.p1.GetIdx(), comp.rh.GetIdx()).GetProp(Property.BOND_OCCUPANCY.value)

'1.89836'

In [13]:
comp.p1.GetPropsAsDict()

{'nbo_charge': 1.1076,
 'nmr_isotropic': 319.369,
 'nmr_anisotropic': 118.7911,
 'molAtomMapNumber': 38,
 'nbo_charge_no_rh': 0.79838,
 'nmr_isotropic_no_rh': 241.6574,
 'nmr_anisotropic_no_rh': 93.1935,
 'lone_pair_occupancy_no_rh_1': 1.93141,
 'lone_pair_energy_no_rh_1': -0.33356}

In [ ]:
df = pd.DataFrame()
param_list = []
for file in os.listdir("./out/spe/"):
    filename = file.split(".")[0]
    ligand_name = filename.replace("_SPE", "")
    path = f"./out/spe/{filename}.out"
    path_no_rh = f"./out/spe_no_rh/{filename}_NoRh.out"
    if not os.path.exists(path_no_rh):
        print(path_no_rh + " does not exist")
        continue
    comp = ParameterExtractor(path=path, no_rh_path=path_no_rh)
    param_dict = {
        "ligand": ligand_name,
        "homo": comp.homo,
        "lumo": comp.lumo,
        "dipole": comp.dipole,
        "rh_charge": comp.rh.GetProp(Property.CHARGE.value),
        "rh_LP_1_occ": comp.rh.GetProp(Property.LONE_PAIR_OCCUPANCY_1.value),
        "rh_LP_1_eng": comp.rh.GetProp(Property.LONE_PAIR_ENERGY_1.value),
        "rh_LP_2_occ": comp.rh.GetProp(Property.LONE_PAIR_OCCUPANCY_2.value),
        "rh_LP_2_eng": comp.rh.GetProp(Property.LONE_PAIR_ENERGY_2.value),
        "rh_LP_3_occ": comp.rh.GetProp(Property.LONE_PAIR_OCCUPANCY_3.value),
        "rh_LP_3_eng": comp.rh.GetProp(Property.LONE_PAIR_ENERGY_3.value),
        "rh_LP_4_occ": comp.rh.GetProp(Property.LONE_PAIR_OCCUPANCY_4.value),
        "rh_LP_4_eng": comp.rh.GetProp(Property.LONE_PAIR_ENERGY_4.value),
        "bite_angle": comp.get_angle(comp.p1, comp.bridge, comp.p2),
        "homo_no_rh": comp.homo_no_rh,
        "lumo_no_rh": comp.lumo_no_rh,
        "dipole_no_rh": comp.dipole_no_rh,
        # missing polarizability, I don't know where to get it from, but does not seem important
    }
    param_list.append(param_dict)
    print(ligand_name + " DONE")
df = pd.DataFrame(param_list)

l_10 DONE
l_11 DONE
l_12 DONE
l_13 DONE
l_14 DONE
l_15 DONE
l_16 DONE
l_17 DONE
l_18 DONE
l_19 DONE
l_1 DONE
l_20b DONE
l_21 DONE
l_22 DONE
l_23 DONE
l_24 DONE
l_25 DONE
l_26 DONE
l_27 DONE
l_28 DONE
l_29 DONE
l_2 DONE
l_30 DONE
l_31 DONE
l_32a DONE


In [21]:
df

,ligand,homo,lumo,dipole,rh_charge,rh_LP_1_occ,rh_LP_1_eng,rh_LP_2_occ,rh_LP_2_eng,rh_LP_3_occ,rh_LP_3_eng,rh_LP_4_occ,rh_LP_4_eng,bite_angle,homo_no_rh,lumo_no_rh,dipole_no_rh
0,l_10,-0.30854,-0.16412,1.1953,-0.34364,1.98349,-0.31041,1.95575,-0.36266,1.79816,-0.32357,1.78943,-0.32714,98.294533,-0.18560,-0.02563,1.5374
1,l_11,-0.31317,-0.16905,0.9229,-0.34856,1.98359,-0.31525,1.95533,-0.36772,1.80166,-0.32999,1.78555,-0.33066,97.829436,-0.18959,-0.03209,0.9771
2,l_12,-0.30549,-0.16352,2.3293,-0.36670,1.98320,-0.30691,1.95379,-0.36021,1.80175,-0.31996,1.79594,-0.32619,97.103496,-0.18398,-0.02436,1.3532
3,l_13,-0.31265,-0.16971,1.7843,-0.36261,1.98181,-0.31413,1.95395,-0.36583,1.80299,-0.32722,1.79435,-0.33069,97.330332,-0.18901,-0.03116,1.1617
4,l_14,-0.30035,-0.15854,3.2379,-0.30737,1.98268,-0.30211,1.95468,-0.35516,1.80823,-0.31759,1.78185,-0.31810,96.503755,-0.18430,-0.05829,1.6744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,l_8,-0.31190,-0.16571,1.6657,-0.35261,1.98477,-0.31359,1.95807,-0.36465,1.80738,-0.32818,1.78242,-0.32740,98.796278,-0.17996,0.01653,1.7227
73,l_90,-0.30978,-0.16505,1.7988,-0.35457,1.97948,-0.31016,1.95435,-0.36466,1.80425,-0.32558,1.78601,-0.32540,97.032439,-0.18018,-0.02270,1.7804
74,l_91,-0.30633,-0.16210,0.4263,-0.35034,1.97891,-0.30601,1.95465,-0.36093,1.80426,-0.32186,1.78426,-0.32178,97.153959,-0.17846,-0.02214,1.8019
75,l_92,-0.30761,-0.16360,1.6455,-0.32650,1.98221,-0.30585,1.95789,-0.36148,1.79773,-0.31803,1.79284,-0.32762,98.598399,-0.18419,-0.02165,1.8320


In [20]:
atoms = comp.mol.GetAtoms()
elements = [a.GetSymbol() for a in atoms]
metal_index = comp.rh.GetIdx() + 1
conformer = comp.mol.GetConformer()
coordinates = [np.array([coord.x, coord.y, coord.z]) for coord in (conformer.GetAtomPosition(a.GetIdx()) for a in atoms)]
BuriedVolume()

In [29]:
coordinates

[array([2.43451e+00, 1.06000e-02, 1.92000e-03]),
 array([ 3.07728, -0.29538, -0.82292]),
 array([3.07171, 0.32266, 0.82881]),
 array([ 1.52523, -1.53962,  2.32611]),
 array([ 2.54889, -1.86612,  2.50961]),
 array([ 0.83447, -2.27426,  2.73786]),
 array([ 1.35422, -0.59262,  2.83688]),
 array([ 1.7007 , -2.92877, -0.32389]),
 array([ 0.79111, -4.04883,  0.20905]),
 array([ 1.03107, -4.98156, -0.30694]),
 array([-0.26586, -3.84309,  0.03295]),
 array([ 0.93438, -4.2251 ,  1.27613]),
 array([ 3.16955, -3.26923, -0.02936]),
 array([ 3.35114, -3.44216,  1.03209]),
 array([ 3.85542, -2.49311, -0.37364]),
 array([ 3.43494, -4.19007, -0.55514]),
 array([ 1.49191, -2.76466, -1.8375 ]),
 array([ 1.72278, -3.70738, -2.33922]),
 array([ 2.1506 , -2.00554, -2.26464]),
 array([ 0.46161, -2.49885, -2.08002]),
 array([ 1.51447,  1.5501 , -2.32507]),
 array([ 2.53558,  1.88494, -2.50786]),
 array([ 0.81806,  2.27851, -2.73836]),
 array([ 1.35191,  0.60119, -2.835  ]),
 array([1.67488, 2.94378, 0.32351]